# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [252]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [253]:
adult_dt

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


In [231]:
type(adult_dt)

pandas.core.frame.DataFrame

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [232]:
from sklearn.model_selection import train_test_split,cross_validate

In [233]:
X = adult_dt.drop(columns = 'income')
Y = adult_dt['income']
train_X,test_X,train_Y,test_Y= train_test_split(X,Y,test_size =.3, random_state=42)

In [234]:
X,Y

(       age          workclass  fnlwgt    education  education-num  \
 0       39          State-gov   77516    Bachelors             13   
 1       50   Self-emp-not-inc   83311    Bachelors             13   
 2       38            Private  215646      HS-grad              9   
 3       53            Private  234721         11th              7   
 4       28            Private  338409    Bachelors             13   
 ...    ...                ...     ...          ...            ...   
 32556   27            Private  257302   Assoc-acdm             12   
 32557   40            Private  154374      HS-grad              9   
 32558   58            Private  151910      HS-grad              9   
 32559   22            Private  201490      HS-grad              9   
 32560   52       Self-emp-inc  287927      HS-grad              9   
 
             marital-status          occupation    relationship    race  \
 0            Never-married        Adm-clerical   Not-in-family   White   
 1      

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)
The random state is used to control the random number generator.
It is useful to get reproducible results. If not specified, the result will be random each time the code is run*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [235]:
from sklearn.preprocessing import OneHotEncoder,RobustScaler
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
num_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


In [236]:
pipe_num = Pipeline([
    ('imputer',KNNImputer(n_neighbors=7,weights='distance')),
    ('standardizer', RobustScaler())
])
pipe_cat = Pipeline(
    [
     ('simple_imputer', SimpleImputer(strategy = 'most_frequent')),
     ('standardizer', OneHotEncoder(handle_unknown='infrequent_if_exist',drop='if_binary'))   
    ]
)
transformer = ColumnTransformer(
    transformers = [
        ('num_transforms',pipe_num,num_cols),
        ('cat_transforms',pipe_cat, cat_cols)
    ], remainder='drop'
)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [237]:
from sklearn.ensemble import RandomForestClassifier
pipe1 = Pipeline(
    [
        ('preproc', transformer),
        ('classifier',RandomForestClassifier())
    ]
)
pipe1

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('standardizer',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('standardizer',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [244]:
from sklearn.metrics import  balanced_accuracy_score
scoring = ['neg_log_loss','roc_auc','accuracy','balanced_accuracy'] 
res_adult_train = cross_validate(pipe1, train_X, train_Y, cv=5, scoring = scoring)
res_adult_test = cross_validate(pipe1, test_X, test_Y, cv=5, scoring = scoring)
res_adult_train, res_adult_test

c:\Users\sarit\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\sarit\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\sarit\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\sarit\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


({'fit_time': array([9.24067926, 9.3684721 , 9.43419838, 8.99449897, 9.04297686]),
  'score_time': array([0.20575476, 0.21020889, 0.20637131, 0.21471667, 0.2212646 ]),
  'test_neg_log_loss': array([-0.3650164 , -0.353487  , -0.39596881, -0.36307699, -0.38591075]),
  'test_roc_auc': array([0.90375641, 0.90250334, 0.90102292, 0.90724034, 0.90324277]),
  'test_accuracy': array([0.85216056, 0.84426409, 0.85234752, 0.85892935, 0.85936814]),
  'test_balanced_accuracy': array([0.77918525, 0.76567589, 0.77485838, 0.78135623, 0.78133815])},
 {'fit_time': array([2.91043782, 2.80723476, 2.8319416 , 2.75338078, 2.79128838]),
  'score_time': array([0.09970164, 0.08820581, 0.09399056, 0.09368134, 0.08772445]),
  'test_neg_log_loss': array([-0.33255453, -0.33304174, -0.33542465, -0.42116544, -0.35234423]),
  'test_roc_auc': array([0.9058056 , 0.90569262, 0.89456972, 0.89082733, 0.89129002]),
  'test_accuracy': array([0.86028659, 0.84800409, 0.84595701, 0.84851586, 0.84639017]),
  'test_balanced_accur

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [245]:
res_val= pd.DataFrame(res_adult_test).assign(experiment = 1).sort_values('test_neg_log_loss')
res_val

,fit_time,score_time,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy,experiment
3,2.753381,0.093681,-0.421165,0.890827,0.848516,0.758525,1
4,2.791288,0.087724,-0.352344,0.891290,0.846390,0.764953,1
2,2.831942,0.093991,-0.335425,0.894570,0.845957,0.759827,1
1,2.807235,0.088206,-0.333042,0.905693,0.848004,0.761913,1
0,2.910438,0.099702,-0.332555,0.905806,0.860287,0.784852,1


Calculate the mean of each metric. 

In [246]:
res_val.mean()

fit_time                  2.818857
score_time                0.092661
test_neg_log_loss        -0.354906
test_roc_auc              0.897637
test_accuracy             0.849831
test_balanced_accuracy    0.766014
experiment                1.000000
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [241]:
res_adult_test = cross_validate(pipe1, test_X, test_Y, cv=5, scoring = scoring)
res_test= pd.DataFrame(res_adult_test).assign(experiment = 1).sort_values('test_neg_log_loss')
res_adult_train,res_adult_test

c:\Users\sarit\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\sarit\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


({'fit_time': array([ 9.72457361,  9.58259153,  9.255476  , 10.54790616,  9.89092612]),
  'score_time': array([0.22594309, 0.20987916, 0.21800232, 0.20606709, 0.22439837]),
  'test_neg_log_loss': array([-0.34102078, -0.38341642, -0.37534149, -0.37075432, -0.37983968]),
  'test_roc_auc': array([0.90608489, 0.90108465, 0.90199708, 0.90691431, 0.90208803]),
  'test_accuracy': array([0.84974775, 0.84689625, 0.85366389, 0.86002633, 0.85892935]),
  'test_balanced_accuracy': array([0.77482693, 0.76741351, 0.777881  , 0.78208024, 0.77643324])},
 {'fit_time': array([3.31749988, 3.39306831, 3.27526975, 3.06683517, 3.34344316]),
  'score_time': array([0.114079  , 0.11521745, 0.11208534, 0.11935163, 0.11960006]),
  'test_neg_log_loss': array([-0.39274636, -0.32978412, -0.40031262, -0.40544594, -0.35170797]),
  'test_roc_auc': array([0.90674501, 0.90772135, 0.89257865, 0.89002264, 0.8919549 ]),
  'test_accuracy': array([0.86386899, 0.85568066, 0.84493347, 0.84698055, 0.84024578]),
  'test_balanced_

In [247]:
from sklearn.metrics import accuracy_score, log_loss, balanced_accuracy_score,roc_auc_score
def evaluate_model(pipe1,train_X, train_Y, test_X, test_Y):
    pipe1.fit(train_X,train_Y)
    Y_pred_train = pipe1.predict(train_X)
    Y_pred_test = pipe1.predict(test_X)
    Y_proba_train = pipe1.predict_proba(train_X)
    Y_proba_test = pipe1.predict_proba(test_X)
    performance_metrics = {
        'log_loss_train': log_loss(train_Y, Y_proba_train),
        'log_loss_test': log_loss(test_Y, Y_proba_test),
        'accuracy_train': accuracy_score(train_Y,Y_pred_train),
        'accuracy_test': accuracy_score(test_Y,Y_pred_test),
        'ROC_AUC_train': roc_auc_score(train_Y,Y_pred_train),
        'ROC_AUC_test': roc_auc_score(test_Y,Y_pred_test),
        'balanced_accuracy_train': balanced_accuracy_score(train_Y,Y_pred_train),
        'balanced_accuracy_test': balanced_accuracy_score(test_Y,Y_pred_test)
         }
    return performance_metrics

In [248]:
evaluate_model(pipe1,train_X, train_Y, test_X, test_Y)

{'log_loss_train': 0.08125751548334942,
 'log_loss_test': 0.3913633602628961,
 'accuracy_train': 0.9998244998244998,
 'accuracy_test': 0.853618589415498,
 'ROC_AUC_train': 0.9996381400398047,
 'ROC_AUC_test': 0.7737101085336564,
 'balanced_accuracy_train': 0.9996381400398047,
 'balanced_accuracy_test': 0.7737101085336565}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)
The classification of income categories '>50k' and below(=<50k>) to numeric values could be useful for:
1. Compatibility and standardization:
Models like logistic regression use 0 and 1 to fit a decision boundary that separates classes. Numbers provide a standardised way of representing categories for easier comparision of models.

2. Performance optimization: Numeric values can be processed more efficiently by computers than categorical data leading to faster computations and more efficient memory usage.

3. Prediction interpretation: The predicted probabilities can be directly interpreted and compared to the numeric target values

4. calculation of metrics : Metrics like accuracy and ROC AUC can be directly computed from numeric values.


## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.